In [1]:
import pandas as pd
import numpy as np

df = (pd.read_csv(r"C:\Users\matte\OneDrive\Desktop\GitHub\data\causal\online_mkt.csv")
              .astype({"date":"datetime64[ns]"})
              .query("post==0"))

df.head()

,app_download,population,city,state,date,post,treated
0,3066.0,12396372,sao_paulo,sao_paulo,2022-03-01,0,1
1,2701.0,12396372,sao_paulo,sao_paulo,2022-03-02,0,1
2,1927.0,12396372,sao_paulo,sao_paulo,2022-03-03,0,1
3,1451.0,12396372,sao_paulo,sao_paulo,2022-03-04,0,1
4,1248.0,12396372,sao_paulo,sao_paulo,2022-03-05,0,1


In [3]:
df_piv = (df.pivot(index="date", columns="city", values="app_download"))

df_piv.head()

city,ananindeua,aparecida_de_goiania,aracaju,belem,belford_roxo,belo_horizonte,brasilia,campinas,campo_grande,campos_dos_goytacazes,...,sao_bernardo_do_campo,sao_goncalo,sao_jose_dos_campos,sao_luis,sao_paulo,serra,sorocaba,teresina,uberlandia,vila_velha
date,,,,,,,,,,,,,,,,,,,,,
2022-03-01,11.0,54.0,65.0,49.0,52.0,194.0,449.0,326.0,305.0,42.0,...,208.0,148.0,173.0,28.0,3066.0,46.0,207.0,68.0,29.0,63.0
2022-03-02,5.0,20.0,42.0,44.0,25.0,130.0,367.0,172.0,261.0,24.0,...,185.0,157.0,137.0,4.0,2701.0,48.0,98.0,17.0,29.0,11.0
2022-03-03,2.0,0.0,0.0,24.0,13.0,112.0,219.0,292.0,171.0,28.0,...,96.0,72.0,171.0,6.0,1927.0,28.0,87.0,55.0,30.0,14.0
2022-03-04,0.0,0.0,11.0,4.0,34.0,56.0,265.0,154.0,175.0,43.0,...,123.0,102.0,0.0,5.0,1451.0,22.0,47.0,49.0,35.0,0.0
2022-03-05,5.0,5.0,0.0,16.0,12.0,45.0,135.0,122.0,189.0,35.0,...,99.0,53.0,112.0,12.0,1248.0,21.0,36.0,31.0,6.0,1.0


In [4]:
f = (df.groupby("city")["population"].first()
             /df.groupby("city")["population"].first().sum())
f[:5]

city
ananindeua              0.007889
aparecida_de_goiania    0.008785
aracaju                 0.009818
belem                   0.021990
belford_roxo            0.007521
Name: population, dtype: float64

In [5]:
y_avg = df_piv.dot(f)
geos = list(df_piv.columns)
n_tr = 5

In [6]:
np.random.seed(1)
rand_geos = np.random.choice(geos, n_tr, replace=False)
rand_geos

array(['manaus', 'recife', 'sao_bernardo_do_campo', 'salvador', 'aracaju'],
      dtype='<U23')